# Lecture 1 - Programming Assignment

## Data Set


In [2]:
pip install pandas

   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ------------------ --------------------- 5.5/11.6 MB 30.5 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 36.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/15.9 MB ? eta -:--:--
   ------------------------------------ --- 14.7/15.9 MB 76.7 MB/s eta 0:00:01
   ---------------------------------------- 15.9/15.9 MB 66.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

# Read one line of data from the CSV file
df = pd.read_csv('oasis_dataset.csv', nrows=1)
print(df)


ModuleNotFoundError: No module named 'pandas'

In [1]:
import warnings

warnings.filterwarnings('ignore')

## Data Set Summary

